In [43]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import StarRocks
from langchain.vectorstores.starrocks import StarRocksSettings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, UnstructuredMarkdownLoader

update_vectordb = False

In [39]:
# load markdown files understand `docs` directory
# for starrocks documents, you can clone repo from https://github.com/StarRocks/starrocks
# and there is `docs` directory in it.

loader = DirectoryLoader('./docs', glob='**/*.md', loader_cls=UnstructuredMarkdownLoader)
documents = loader.load()

In [40]:
# load text splitter and split docs into snippets of text
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)

# tell vectordb to update text embeddings
update_vectordb = True

Created a chunk of size 2318, which is longer than the specified 600
Created a chunk of size 1391, which is longer than the specified 600
Created a chunk of size 723, which is longer than the specified 600
Created a chunk of size 966, which is longer than the specified 600
Created a chunk of size 1309, which is longer than the specified 600
Created a chunk of size 981, which is longer than the specified 600
Created a chunk of size 1078, which is longer than the specified 600
Created a chunk of size 720, which is longer than the specified 600
Created a chunk of size 613, which is longer than the specified 600
Created a chunk of size 659, which is longer than the specified 600
Created a chunk of size 1600, which is longer than the specified 600
Created a chunk of size 4034, which is longer than the specified 600
Created a chunk of size 4036, which is longer than the specified 600
Created a chunk of size 1118, which is longer than the specified 600
Created a chunk of size 833, which is lo

Created a chunk of size 1037, which is longer than the specified 600
Created a chunk of size 684, which is longer than the specified 600
Created a chunk of size 656, which is longer than the specified 600
Created a chunk of size 692, which is longer than the specified 600
Created a chunk of size 605, which is longer than the specified 600
Created a chunk of size 776, which is longer than the specified 600
Created a chunk of size 830, which is longer than the specified 600
Created a chunk of size 768, which is longer than the specified 600
Created a chunk of size 842, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 815, which is longer than the specified 600
Created a chunk of size 822, which is longer than the specified 600
Created a chunk of size 727, which is longer than the specified 600
Created a chunk of size 785, which is longer than the specified 600
Created a chunk of size 704, which is longer th

Created a chunk of size 692, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 612, which is longer than the specified 600
Created a chunk of size 3319, which is longer than the specified 600
Created a chunk of size 612, which is longer than the specified 600
Created a chunk of size 727, which is longer than the specified 600
Created a chunk of size 615, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 1059, which is longer than the specified 600
Created a chunk of size 937, which is longer than the specified 600
Created a chunk of size 622, which is longer than the specified 600
Created a chunk of size 809, which is longer than the specified 600
Created a chunk of size 997, which is longer than the specified 600
Created a chunk of size 1008, which is longer 

Created a chunk of size 853, which is longer than the specified 600
Created a chunk of size 824, which is longer than the specified 600
Created a chunk of size 982, which is longer than the specified 600
Created a chunk of size 679, which is longer than the specified 600
Created a chunk of size 657, which is longer than the specified 600
Created a chunk of size 2401, which is longer than the specified 600
Created a chunk of size 4749, which is longer than the specified 600
Created a chunk of size 658, which is longer than the specified 600
Created a chunk of size 1036, which is longer than the specified 600
Created a chunk of size 1933, which is longer than the specified 600
Created a chunk of size 1262, which is longer than the specified 600
Created a chunk of size 1582, which is longer than the specified 600
Created a chunk of size 2244, which is longer than the specified 600
Created a chunk of size 2244, which is longer than the specified 600
Created a chunk of size 1906, which is l

In [41]:
print('# docs  = %d, # splits = %d' % (len(documents), len(split_docs)))

# docs  = 657, # splits = 5179


In [42]:
# use chroma as vectordb
def gen_chroma(update_vectordb, embeddings, persist_directory = './vectordb'):
    if update_vectordb:
        docsearch = Chroma.from_documents(split_docs, embeddings, persist_directory = persist_directory)        
    else:
        docsearch = Chroma(persist_directory = persist_directory, embedding_function=embeddings)
    return docsearch

# use starrocks as vectordb
def gen_starrocks(update_vectordb, embeddings, settings):
    if update_vectordb:
        docsearch = StarRocks.from_documents(split_docs, embeddings, config = settings)            
    else:
        docsearch = StarRocks(embeddings, settings)    
    return docsearch

In [44]:
embeddings = OpenAIEmbeddings()

# configure starrocks settings(host/port/user/pw/db)
settings = StarRocksSettings()
settings.port = 41003
settings.host = '127.0.0.1'
settings.username = 'root'
settings.password = ''
settings.database = 'zya'

# docsearch = gen_chroma(embeddings)
docsearch = gen_starrocks(update_vectordb, embeddings, settings)
update_vectordb = False

In [45]:
llm = OpenAI()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
query = "how to enable query profile"
resp = qa.run(query)
print(resp)

 Set the variable `enable_profile` to `true`.
